# Build a Local RAG

## Setup

In [1]:
import os

os.chdir("../../../")

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

## Document Loading, Embedding, and Model

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=local_embeddings)

In [6]:
question = "What are the approaches to Task Decomposition?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [8]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [9]:
response_message = model.invoke(
    "How big is Maringá?"
)

print(response_message.content)

Maringá is a city located in the state of Paraná, Brazil. As for its size, I can provide you with some information:

* Population: According to the 2020 census, Maringá has a population of approximately 396,506 people.
* Area: The municipality of Maringá covers an area of about 552 square kilometers (213 sq mi).

To give you a better idea, here are the city's dimensions:

* Length: around 32 km (20 miles) from north to south
* Width: around 18 km (11 miles) from east to west

Maringá is considered one of the largest cities in Paraná state and an important economic center in the region.


## Using in a Chain

We can create a summarization chain with either model by passing in retrieved docs and a simple prompt.

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)


'The main themes in these retrieved documents are:\n\n1. **Task Decomposition**: Breaking down complex tasks into smaller, manageable subgoals using various methods such as:\n\t* Using Large Language Models (LLMs) with simple prompting\n\t* Providing task-specific instructions\n\t* Receiving human input\n2. **Planning and Execution**: An overview of an LLM-powered autonomous agent system, which involves:\n\t* Planning: Breaking down tasks into subgoals and planning ahead\n\t* Task Execution: Expert models executing specific tasks and logging results\n3. **Reflection and Refinement**: The importance of self-criticism, self-reflection, learning from mistakes, and refining actions for future steps to improve the quality of final results.'

In [11]:
docs[0].page_content

'Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.'

## Q&A

You can also perform question-answering with your local model and vector store. Here's an example with a simple string prompt:

In [13]:
from langchain_core.runnables import RunnablePassthrough

RAG_TEMPLATE = """
You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt 
    | model
    | StrOutputParser()
)


question = "What are the approaches to Task Decomposition?"

docs = vectorstore.similarity_search(question)

# run
chain.invoke({"context": docs, "question": question})

'There are three approaches to task decomposition: (1) using Large Language Models (LLM) with simple prompting, (2) applying task-specific instructions, and (3) incorporating human inputs. This process breaks down large tasks into smaller, manageable subgoals for efficient handling of complex tasks. It enables the agent to plan ahead and execute tasks more effectively.'

## Q&A with Retrieval

Instead of manually passing in docs, you can automatically retrieve them from our vector store based on the user question:

In [14]:
retriever = vectorstore.as_retriever()

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser()
)

question = "What are the approaches to Task Decomposition"

qa_chain.invoke(question)

'Task decomposition can be done in three ways: (1) using simple prompting like "Steps for XYZ", (2) using task-specific instructions, and (3) with human inputs. Additionally, it can also be achieved through task execution by expert models executing on specific tasks.'